In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession\
        .builder.appName('RDS Connection')\
        .config('spark.jars','mysql-connector-java-8.0.25.jar')\
        .getOrCreate()

In [3]:
%%time

mysql_select_query='(select userId,loanId from userloan)as userloan'

mysql_jdbc_url='jdbc:mysql://mydb.comrcebmnhti.us-east-2.rds.amazonaws.com:3306/coddletech'

tbl_data = spark.read.format('jdbc')\
            .option('url',mysql_jdbc_url)\
            .option('driver','com.mysql.jdbc.Driver')\
            .option('dbtable',mysql_select_query)\
            .option('user','admin')\
            .option('password','1996Aswin')\
            .load()

tbl_data.show(5)

+--------------------+---------+
|              userId|   loanId|
+--------------------+---------+
|06ebc429-8987-478...|MX-2582-1|
|06ebc429-8987-478...|MX-2582-2|
|00335c42-70c0-416...| MX-368-3|
|00335c42-70c0-416...| MX-368-1|
|00335c42-70c0-416...| MX-368-2|
+--------------------+---------+
only showing top 5 rows

Wall time: 23.9 s


In [4]:
loan_count=tbl_data.groupby('userId').count().withColumnRenamed('count','total_loan').groupBy('total_loan').count()
loan_count=loan_count.orderBy('total_loan')
loan_count.show()

+----------+-----+
|total_loan|count|
+----------+-----+
|         1|  449|
|         2|  327|
|         3|  223|
|         4|  112|
|         5|   55|
|         6|   27|
|         7|    9|
|         8|    6|
|         9|    6|
|        10|    1|
|        11|    1|
|        13|    1|
+----------+-----+



In [ ]:
loan_count.write.format('jdbc')\
            .option('url',mysql_jdbc_url)\
            .option('driver','com.mysql.jdbc.Driver')\
            .option('dbtable','loan_count')\
            .option('user','admin')\
            .option('mode','append')\
            .option('password','1996Aswin')\
            .save()